# Import Lib

In [1]:
import open3d as o3d
import numpy as np

import pandas
from sklearn.decomposition import PCA

import copy

In [2]:
# add aixs
aix_points = [[0, 0, 0],
              [0, 0, 100],
              [400, 0, 0],
              [0, 100, 0],
              [-400, 0, 0],]
aix_lines = [[0, 1],
             [0, 2],
             [0, 3],
             [2,4]]
colors = [[0, 0, 0] for i in range(len(aix_lines))]
aix_line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(aix_points),
    lines=o3d.utility.Vector2iVector(aix_lines),
)
aix_line_set.colors = o3d.utility.Vector3dVector(colors)

# Read scan model

## import obj file

In [3]:
scan_obj = o3d.io.read_triangle_mesh("./data/femur_half_3.obj")
print(scan_obj)

geometry::TriangleMesh with 35943 points and 70305 triangles.


In [4]:
o3d.visualization.draw_geometries([scan_obj], mesh_show_wireframe=True)

# Scale unit length to 1mm (coordinate 1000x)

## point center

In [5]:
points_center = scan_obj.get_center()
print(points_center)

[ 0.24965531 -0.49856483 -0.2488121 ]


In [6]:
scan_obj.scale(1000.0, points_center)

geometry::TriangleMesh with 35943 points and 70305 triangles.

## save file

# Delete floor plane

## change to point cloud

In [7]:
number_of_points = np.asarray(scan_obj.vertices).shape[0]

In [8]:
scan_obj.compute_vertex_normals()
scan_pcd = scan_obj.sample_points_uniformly(number_of_points)
# o3d.visualization.draw_geometries([scan_pcd])

## find plane using RANSAC

### plane function: ax + by + cz + d = 0

In [9]:
plane_model, inliers = scan_pcd.segment_plane(distance_threshold=5,
                                              ransac_n=3,
                                              num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

Plane equation: 0.00x + 1.00y + -0.00z + 5.63 = 0


In [10]:
# floor
inlier_cloud = scan_pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])

# bone 
bone_cloud = scan_pcd.select_by_index(inliers, invert=True)

o3d.visualization.draw_geometries([inlier_cloud, bone_cloud])

# Delete outliers

In [11]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [12]:
cl, ind = bone_cloud.remove_statistical_outlier(nb_neighbors=10,
                                                std_ratio=2)
# display outliers
display_inlier_outlier(bone_cloud, ind)

Showing outliers (red) and inliers (gray): 


In [13]:
bone_cloud = bone_cloud.select_by_index(ind)
# o3d.visualization.draw_geometries([bone_cloud])

### write point cloud to file

In [14]:
# o3d.io.write_point_cloud("./data/femur_half_4_bone_point_cloud.ply", bone_cloud)

# Read Target model

## read mesh from target model

In [15]:
target_obj = o3d.io.read_triangle_mesh("./data/Femur.stl")
print(target_obj)

geometry::TriangleMesh with 10415430 points and 3471810 triangles.


### down sample to point clould

In [16]:
number_of_points = np.asarray(bone_cloud.points).shape[0]
print(number_of_points)

4644


In [17]:
target_pcd = target_obj.sample_points_uniformly(number_of_points*3, seed=10)
# o3d.visualization.draw_geometries([target_pcd], mesh_show_wireframe=True)

In [18]:
# o3d.io.write_point_cloud("./data/femur_target_point_cloud.ply", target_pcd)

# Calculate PCA

## Target

### subtract mean each column

In [19]:
target_bone_points_cloud_array = np.asarray(target_pcd.points)

In [20]:
target_bone_points_cloud_array = target_bone_points_cloud_array - target_bone_points_cloud_array.mean(axis=0, keepdims=True)
print(target_bone_points_cloud_array)

[[ -90.25784117   23.95714346 -180.40704772]
 [ -89.39301092   23.93136748 -180.08638777]
 [ -90.21689171   25.87124719 -180.23820867]
 ...
 [  82.89677361  -13.94142222  204.11958784]
 [  84.921167    -14.35868464  204.15772746]
 [  82.03451461  -15.02256935  204.12778006]]


### PCA

In [21]:
target_pca = PCA(n_components=3)
target_pca.fit(target_bone_points_cloud_array)
print(target_pca.explained_variance_ratio_)

[0.97480933 0.01605089 0.00913978]


In [22]:
target_bone_points_cloud_array = target_pca.transform(target_bone_points_cloud_array)

In [23]:
target_bone_pca_pcd = o3d.geometry.PointCloud()
target_bone_pca_pcd.points = o3d.utility.Vector3dVector(target_bone_points_cloud_array)

In [24]:
# o3d.visualization.draw_geometries([target_bone_pca_pcd], mesh_show_wireframe=True)

## Scan

## subtract mean each column

In [25]:
scan_bone_points_cloud_array = np.asarray(bone_cloud.points)

In [26]:
scan_bone_points_cloud_array = scan_bone_points_cloud_array - scan_bone_points_cloud_array.mean(axis=0, keepdims=True)
print(scan_bone_points_cloud_array)

[[188.45149449   3.23794028  46.36757462]
 [188.43364571   5.27543925  43.60877148]
 [185.99696224   3.59406919  43.99422632]
 ...
 [ 17.65011234 -25.47040629 -17.36179282]
 [ 30.45557492 -25.86759004 -19.01281202]
 [ 34.2867115  -25.39438481 -19.24860018]]


### PCA

In [27]:
scan_pca = PCA(n_components=3)
scan_pca.fit(scan_bone_points_cloud_array)
print(scan_pca.explained_variance_ratio_)

[0.97283093 0.01911746 0.00805161]


In [28]:
scan_bone_points_cloud_array = scan_pca.transform(scan_bone_points_cloud_array)

In [29]:
scan_bone_pca_pcd = o3d.geometry.PointCloud()
scan_bone_pca_pcd.points = o3d.utility.Vector3dVector(scan_bone_points_cloud_array)

In [30]:
o3d.visualization.draw_geometries([scan_bone_pca_pcd], mesh_show_wireframe=True)

In [31]:
o3d.visualization.draw_geometries([scan_bone_pca_pcd, aix_line_set], mesh_show_wireframe=True)

## visualize together

In [32]:
target_bone_pca_pcd.paint_uniform_color([1, 0, 0])
scan_bone_pca_pcd.paint_uniform_color([0, 1, 0])

geometry::PointCloud with 4644 points.

In [33]:
o3d.io.write_point_cloud("./data/target_temp.ply", target_bone_pca_pcd)
o3d.io.write_point_cloud("./data/scan_temp.ply", scan_bone_pca_pcd)

True

In [34]:
o3d.visualization.draw_geometries([target_bone_pca_pcd, scan_bone_pca_pcd,aix_line_set])

# Registration

## define source and target, helper plot fucntion

In [35]:
source = scan_bone_pca_pcd
target = target_bone_pca_pcd

In [36]:
def draw_registration_result(source, target, transformation, name):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([0, 1, 0])
    target_temp.paint_uniform_color([1, 0, 0])
    source_temp.transform(transformation)
    
    # add aixs
    aix_points = [[0, 0, 0],
                  [0, 0, 100],
                  [100, 0, 0],
                  [0, 100, 0]]
    aix_lines = [[0, 1],
                 [0, 2],
                 [0, 3]]
    colors = [[0, 0, 0] for i in range(len(aix_lines))]
    aix_line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(aix_points),
        lines=o3d.utility.Vector2iVector(aix_lines),
    )
    aix_line_set.colors = o3d.utility.Vector3dVector(colors)
    
    o3d.visualization.draw_geometries([source_temp, target_temp, aix_line_set], name)

## initial transformation matrix

In [37]:
source_array = np.asarray(source.points)
target_array = np.asarray(target.points)

In [38]:
source_max_in_columns = np.amax(source_array, axis=0)
target_max_in_columns = np.amax(target_array, axis=0)

source_min_in_columns = np.amin(source_array, axis=0)
target_min_in_columns = np.amin(target_array, axis=0)

In [39]:
print('source:')
print(source_min_in_columns)
print(source_max_in_columns)

print('target: ')
print(target_min_in_columns)
print(target_max_in_columns)

source:
[-217.1434739   -41.51935731  -28.53159097]
[221.44780156  48.03843825  30.2976717 ]
target: 
[-203.99948496  -34.39841836  -24.4194598 ]
[221.19629749  48.75596655  37.88527186]


In [40]:
# strech = np.divide(target_max_in_columns - target_min_in_columns, source_max_in_columns - source_min_in_columns) 
# print(strech)

[0.96973729 0.92757486 1.06890979]


In [48]:
threshold =20
# trans_init = np.asarray([[strech[0], 0, 0, 0],
#                          [0, strech[1], 0, 0],
#                          [0, 0, strech[2], 0],
#                          [0, 0,  0, 1]])
trans_init = np.asarray([[1, 0, 0, 0],
                         [0, 1, 0, 0],
                         [0, 0, 1, 0],
                         [0, 0, 0, 1]])
draw_registration_result(source, target, trans_init, 'init matrix result')

In [49]:
evaluation = o3d.registration.evaluate_registration(source, target, threshold, trans_init)
print(evaluation)

registration::RegistrationResult with fitness=1.000000e+00, inlier_rmse=5.847886e+00, and correspondence_set size of 4644
Access transformation to get result.


In [50]:
reg_p2p = o3d.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.registration.TransformationEstimationPointToPoint(),
        o3d.registration.ICPConvergenceCriteria(max_iteration = 2000))
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)

registration::RegistrationResult with fitness=1.000000e+00, inlier_rmse=3.948725e+00, and correspondence_set size of 4644
Access transformation to get result.
Transformation is:
[[ 9.99995104e-01  4.90953404e-04  3.09038194e-03  5.73745126e+00]
 [-9.22857922e-04  9.89959063e-01  1.41351341e-01  5.22775991e-01]
 [-2.98995469e-03 -1.41353501e-01  9.89954670e-01  5.68339299e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [51]:
# draw_registration_result(source, target, reg_p2p.transformation, 'p2p')
# compute cpd registration
result = copy.deepcopy(source)
result = result.transform(reg_p2p.transformation)

# draw result: source: green, target: red, registed: blue
source.paint_uniform_color([0, 1, 0])
target.paint_uniform_color([1, 0, 0])
result.paint_uniform_color([0, 0, 1])
o3d.visualization.draw_geometries([source, target, result])

In [45]:
o3d.io.write_point_cloud("./data/result_temp.ply", result)

True

# -------------------------------------------------

In [ ]:
# from probreg import cpd

In [ ]:
# tf_param, _, _ = cpd.registration_cpd(source, target)

In [ ]:
# # compute cpd registration
# result = copy.deepcopy(source)
# result.points = tf_param.transform(result.points)

# # draw result
# source.paint_uniform_color([0, 1, 0])
# target.paint_uniform_color([1, 0, 0])
# result.paint_uniform_color([0, 0, 1])
# o3d.visualization.draw_geometries([source, target, result])